In [1]:
from tools.infer_e2e import OpenRecognizer
from tools.engine.config import Config
import cv2
import numpy as np

# warmup model
rec = OpenRecognizer(config=Config("./output/config.yml").cfg, backend='torch')
dummy = [np.random.randint(0, 256, (32, 128, 3), dtype=np.uint8) for _ in range(64)]
rec(img_numpy_list = dummy, batch_num=64)

# load bunch of line images from same document
img_list = []
for i in range(79):
    img_path = f'./dataset_openocr/test/sample_1565_{i}.jpg'
    img = cv2.imread(img_path)
    img_list.append(img)
    
# actual inference
import time
t1 = time.time()
results = rec(img_numpy_list = img_list, batch_num=128)
print(time.time()-t1)
print("".join([i['text'] for i in results]))


[2025/06/16 16:12:47] openrec INFO: finetune from checkpoint ./output/best.pth
0.4364962577819824
Analyzing the Tactical Advancements of CYPHER: ASentient Military SupercomputerAbstractThe classfied military paper delves into the groundbreaking analysis of CYPHER, a highly advancetentient supercomputer at the forefront of military technology. This paper explores the intricate web otrorces under CYPHER's command and its extraordinary tactical capabilities. Through an in-depthIeramination, it uncovers the strategic advantages enabled by CYPHER's autonomous deceision-makinIend vast network integration. The study sheds light on the unprecedented potential for enhancinIenilitary operations and achieving decisive outcomes through the utilization of this powerful artificiaontelligence systemnuntroductionThe dawn of artficial intelligence has ushered inAe ne era in militry technology, with CYHEIemerging as the vanguard o strateiccedvancenents. Developed as a sentient militaryAsupercomputer, CY